In [1]:
using BenchmarkTools
using PyCall
using Conda
using Libdl
using StatsPlots
using Plots
using Printf

gr()
Plots.GRBackend()


Plots.GRBackend()

In [2]:
a = rand(10^7) ;# Vetor unidimensional com número aleatórios.
sum(a)

d = Dict()

Dict{Any, Any}()

## C language

In [3]:

C_code = """
#include <stddef.h>
double c_sum(size_t n, double *X) {
    double s = 0.0;
    for (size_t i = 0; i < n; ++i) {
        s += X[i];
    }
    return s;
}
"""

const Clib = tempname()  

open(`gcc -fPIC -O3 -msse3 -xc -shared -o $(Clib * "." * Libdl.dlext) -`, "w") do f
    print(f, C_code) 
end

# definindo a função em Julia que chama a função em C :
c_sum(X::Array{Float64}) = ccall(("c_sum", Clib), Float64, (Csize_t, Ptr{Float64}), length(X), X)

c_bench = @benchmark c_sum($a)

d["C"] = median(c_bench.times) / 1e6  # in milliseconds

10.74062

## C com a biblioteca -ffast-math 

In [4]:
const Clib_fastmath = tempname()   # make a temporary file

open(`gcc -fPIC -O3 -msse3 -xc -shared -ffast-math -o $(Clib_fastmath * "." * Libdl.dlext) -`, "w") do f
    print(f, C_code) 
end

c_sum_fastmath(X::Array{Float64}) = ccall(("c_sum", Clib_fastmath), Float64, (Csize_t, Ptr{Float64}), length(X), X)

c_fastmath_bench = @benchmark $c_sum_fastmath($a)

d["C ffast-math"] = median(c_fastmath_bench.times)/1e6

7.286591

## Python Built-in

In [5]:
pysum = pybuiltin("sum")

py_list_bench = @benchmark $pysum($a)

d["Python- Built-in"] = median(py_list_bench.times)/1e6

1631.205901

## Python com **Numpy**

In [6]:


numpy_sum = pyimport("numpy")["sum"]

py_numpy_bench = @benchmark $numpy_sum($a)

d["Python numpy"] = median(py_numpy_bench.times) / 1e6

4.745711

## Python definindo a função

In [7]:
py"""
def py_sum(A):
    s = 0.0
    for a in A:
        s += a
    return s
"""

sum_py = py"py_sum"

py_hand = @benchmark $sum_py($a)

d["Python hand-written"] = median(py_hand.times) / 1e6


1942.861377

## Julia Built-in

In [8]:
j_bench = @benchmark sum($a)
d["Julia built-in"] = median(j_bench.times) / 1e6


6.45389

## Julia definindo a função

In [9]:
function mysum(A)   
    s = 0.0 # s = zero(eltype(a))
    for a in A
        s += a
    end
    s
end

j_bench_hand = @benchmark mysum($a)

d["Julia hand-written"] = median(j_bench_hand.times) / 1e6

10.7421005

## Resultados

In [10]:
for (key, value) in sort(collect(d), by=last)
    println(rpad(key, 30, "."), lpad(round(value; digits=1), 6, "."), "ms")
end

Python numpy.....................4.7ms
Julia built-in...................6.5ms
C ffast-math.....................7.3ms
C...............................10.7ms
Julia hand-written..............10.7ms
Python- Built-in..............1631.2ms
Python hand-written...........1942.9ms


In [54]:
l_vec = sort(collect(d), by=x->x[2], rev = true)

x = map(x->x[1], l_vec)
y = map(x->x[2], l_vec)

y_annotations = [@sprintf(" %.2f ms",i) for i in y]  

Plots.bar(x ,y ,
    xaxis = ("time (ms)",false, nothing), 
    yaxis = ("Languages",false),
    orientation= :horizontal,label = "" , series_annotation = (y_annotations, font(7, :left)),right_margin = 12Plots.mm)


png("howdy")

In [49]:
d

Dict{Any, Any} with 7 entries:
  "C"                   => 10.7406
  "C ffast-math"        => 7.28659
  "Python- Built-in"    => 1631.21
  "Python numpy"        => 4.74571
  "Python hand-written" => 1942.86
  "Julia built-in"      => 6.45389
  "Julia hand-written"  => 10.7421